In [2]:
# !pip install shap
# !pip install catboost
# !pip install optuna
# !pip install lightgbm

In [3]:
import pandas as pd
from lightgbm import LGBMRegressor
import optuna
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization import plot_optimization_history
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,\
mean_squared_log_error
from catboost import CatBoostRegressor
import shap
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
import warnings
warnings. filterwarnings("ignore")
RAND = 10
N_FOLDS = 3

In [4]:

def r2_adjusted(y_true: np.ndarray, y_pred: np.ndarray,X_test: np.ndarray) -> float:
    """Коэффициент детерминации (множественная регрессия)"""
    N_objects = len(y_true)
    N_features = X_test.shape[1]
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (N_objects - 1) / (N_objects - N_features - 1)

def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean percentage error"""
    return np.mean((y_true - y_pred) / y_true) * 100

def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean absolute percentage error"""
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100
def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Weighted Absolute Percent Error"""
    return np.sum(np.abs(y_pred - y_true)) / np.sum(y_true) * 100
def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    """The Root Mean Squared Log Error (RMSE) metric
    Логарифмическая ошибка средней квадратичной ошибки"""
    try:
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    except:
        return None
def get_metrics(y_test: np.ndarray,y_pred: np.ndarray,X_test: np.ndarray,name: str = None):
    """Генерация таблицы с метриками"""
    df_metrics = pd.DataFrame()
    df_metrics['model']=[name]
    df_metrics['MAE'] = mean_absolute_error(y_test, y_pred)
    df_metrics['MSE'] = mean_squared_error(y_test, y_pred)
    df_metrics['RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
    df_metrics['RMSE'] = rmsle(y_test, y_pred)
    df_metrics['R2 adjusted'] = r2_adjusted(y_test, y_pred,X_test)
    df_metrics['MPE_%'] = mpe(y_test, y_pred)
    df_metrics['MAPS_%'] = mape(y_test, y_pred)
    df_metrics['WAPE_%'] = wape(y_test, y_pred)
    
    return df_metrics
    
def check_overfitting(model, X_train, y_train, X_test, y_test, metric_fun):
    """Проверка на overfitting"""
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    mae_train = metric_fun(y_train, y_pred_train)
    mae_test = metric_fun(y_test, y_pred_test)
    print(f'{metric_fun.__name__} train: %.3f' % mae_train)
    print(f'{metric_fun.__name__} test: %.3f' % mae_test)
    print(f'delta = {(abs(mae_train - mae_test)/mae_train*100):.1f} %')

df = pd.read_excel('clear_data.xlsx')
df = df.drop('Номер объявления', axis=1)
cols_cat = df.select_dtypes('object').columns
df[cols_cat] = df[cols_cat].astype('category')


1. Загрузка данных
df - Информация о квартирах

Наименование признаков

Номер объявления
Тип продавца
Цена
Район
Количество комнат
Общая площадь
Этаж
Этажей в доме
Санузел
Ремонт
Вид сделки
Пассажирский лифт

In [5]:
df = pd.read_excel('clear_data.xlsx')

In [6]:
df = df.drop('Номер объявления', axis=1)
cols_cat = df.select_dtypes('object').columns
df[cols_cat] = df[cols_cat].astype('category')

In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2078 entries, 0 to 2077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Тип продавца       2078 non-null   category
 1   Цена               2078 non-null   int64   
 2   Район              2078 non-null   category
 3   Количество комнат  2078 non-null   int64   
 4   Общая площадь      2078 non-null   int64   
 5   Этаж               2078 non-null   int64   
 6   Этажей в доме      2078 non-null   int64   
 7   Санузел            2078 non-null   category
 8   Ремонт             2078 non-null   category
 9   Вид сделки         2078 non-null   category
 10  Пассажирский лифт  2078 non-null   int64   
dtypes: category(5), int64(6)
memory usage: 108.5 KB


In [8]:
X = df.drop(['Цена'], axis=1) 
y = df['Цена']


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.25,
                                                    random_state=101,
                                                    shuffle=True)

X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.16,
                                                  random_state=101,
                                                  shuffle=True)

eval_set = [(X_val, y_val)]

1.1.1 Baseline

In [9]:
from sklearn.compose import ColumnTransformer
transformers_list = [
                    ('encode', OneHotEncoder(dtype='int', drop='first'), [
                    'Тип продавца',
                    'Район',
                    'Санузел', 'Ремонт', 'Вид сделки'
                    ]),
                    ('scale', StandardScaler(),
                    ['Количество комнат', 'Общая площадь', 'Этаж', 'Этажей в доме',  'Пассажирский лифт'])]
column_transformer = ColumnTransformer(transformers_list)

In [10]:
column_transformer

ColumnTransformer(transformers=[('encode',
                                 OneHotEncoder(drop='first', dtype='int'),
                                 ['Тип продавца', 'Район', 'Санузел', 'Ремонт',
                                  'Вид сделки']),
                                ('scale', StandardScaler(),
                                 ['Количество комнат', 'Общая площадь', 'Этаж',
                                  'Этажей в доме', 'Пассажирский лифт'])])

In [11]:
pipe = Pipeline([('columnTransformer', column_transformer),
                ('lr', LinearRegression())])
pipe.fit(X_train, y_train)

      

Pipeline(steps=[('columnTransformer',
                 ColumnTransformer(transformers=[('encode',
                                                  OneHotEncoder(drop='first',
                                                                dtype='int'),
                                                  ['Тип продавца', 'Район',
                                                   'Санузел', 'Ремонт',
                                                   'Вид сделки']),
                                                 ('scale', StandardScaler(),
                                                  ['Количество комнат',
                                                   'Общая площадь', 'Этаж',
                                                   'Этажей в доме',
                                                   'Пассажирский лифт'])])),
                ('lr', LinearRegression())])

In [12]:
y_pred = pipe.predict(X_test)

In [13]:
metrics = get_metrics(y_test, y_pred, X_test, 'Linear Regression_baseline')
metrics

,model,MAE,MSE,RMSE,R2 adjusted,MPE_%,MAPS_%,WAPE_%
0,Linear Regression_baseline,942747.984827,1.877517e+12,0.233705,0.715686,-4.783434,17.739727,14.98419


In [14]:
check_overfitting(pipe,
X_train,
y_train,
X_test,
y_test,
metric_fun=mean_absolute_error)

mean_absolute_error train: 969849.645
mean_absolute_error test: 942747.985
delta = 2.8 %


In [15]:
lgb = LGBMRegressor(random_state=RAND, objective='mae')
lgb.fit(X_train_,
y_train_,
eval_metric="mae",
eval_set=eval_set)
y_pred = lgb.predict(X_test)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 145
[LightGBM] [Info] Number of data points in the train set: 1308, number of used features: 9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 5800000.000000


In [16]:
new_metrics =get_metrics(y_test, y_pred, X_test, 'LGBMRegressor_baseline')
metrics = pd.concat([metrics, new_metrics], ignore_index=True)
metrics


,model,MAE,MSE,RMSE,R2 adjusted,MPE_%,MAPS_%,WAPE_%
0,Linear Regression_baseline,942747.984827,1.877517e+12,0.233705,0.715686,-4.783434,17.739727,14.984190
1,LGBMRegressor_baseline,862257.812614,1.744477e+12,0.196221,0.735832,-1.932412,14.320268,13.704866


In [18]:
check_overfitting(lgb,
X_train,
y_train,
X_test,
y_test,
metric_fun=mean_absolute_error)

mean_absolute_error train: 616427.164
mean_absolute_error test: 862257.813
delta = 39.9 %


 MAE для обучения: 616427,164 является относительно низким показателем, что указывает на то, 
    что модель хорошо предсказывает целевую переменную на обучающих данных.
 MAE для теста: 862257,813 является более высоким показателем, что указывает на то,
    что модель не так хорошо предсказывает целевую переменную на тестовых данных.
 Дельта: Дельта между MAE для обучения и MAE для теста составляет 39,9%, что указывает на то, что модель переобучается.

Возможные причины переобучения:
 Модель может быть слишком сложной для данных.
 Данные могут содержать шум или выбросы, которые могут искажать результаты модели.
 Метрика MAE может быть не подходящей для оценки эффективности модели для конкретной задачи.
 по моему мнению мало данных для такой сложной модели.

1.1.1.2 CatBoostRegressor

In [19]:
cat_features = X_val.select_dtypes('category').columns.tolist()
cb = CatBoostRegressor(random_seed=RAND,
                        loss_function='MAE',
                        eval_metric='MAE',
                        cat_features=cat_features,
                        allow_writing_files=False)

eval_set = [(X_val, y_val)]
cb.fit(X_train_, y_train_,eval_set=eval_set,verbose=0,early_stopping_rounds=100)
y_pred_cb = cb.predict(X_test)

In [20]:
new_metrics =get_metrics(y_test, y_pred_cb, X_test, 'CatBoostRegressor_baseline')
metrics = pd.concat([metrics, new_metrics], ignore_index=True)

In [21]:
metrics

,model,MAE,MSE,RMSE,R2 adjusted,MPE_%,MAPS_%,WAPE_%
0,Linear Regression_baseline,942747.984827,1.877517e+12,0.233705,0.715686,-4.783434,17.739727,14.984190
1,LGBMRegressor_baseline,862257.812614,1.744477e+12,0.196221,0.735832,-1.932412,14.320268,13.704866
2,CatBoostRegressor_baseline,824528.771440,1.501729e+12,0.184894,0.772592,-2.274080,13.723411,13.105195


In [22]:
check_overfitting(cb,
X_train,
y_train,
X_test,
y_test,
metric_fun=mean_absolute_error)

mean_absolute_error train: 672865.208
mean_absolute_error test: 824528.771
delta = 22.5 %
